In [1]:
import sys
import os
import glob
import tensorflow as tf
import numpy as np
sys.path.append("/".join(os.getcwd().split("/")[:-1]))
#tf.InteractiveSession()
tf.enable_eager_execution()

/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/az01640/P

In [2]:
from src.utils.audio import load_processed_audio

In [3]:
positives, negatives, backgrounds = load_processed_audio()

W0821 13:51:00.622977 140734821463488 deprecation_wrapper.py:119] From /Users/az01640/Projets/multrigger-word/src/utils/audio.py:40: The name tf.read_file is deprecated. Please use tf.io.read_file instead.

W0821 13:51:01.993330 140734821463488 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0821 13:51:02.010198 140734821463488 deprecation.py:323] From /Users/az01640/Projets/multrigger-word/src/utils/audio.py:42: decode_audio (from tensorflow.contrib.ffmpeg.ffmpeg_ops) is deprecated and will be removed after 2018-09-04.
Instructions for updating:
tf.contrib.ffmpeg will be removed in 2.0, the support for video and audio will continue to be provided in tens

In [4]:
from src.train import *

In [5]:
ds = (tf.data.Dataset.from_generator(background_generator(backgrounds, 10*16000), output_types=tf.float32))

W0821 13:51:59.167674 140734821463488 deprecation.py:323] From /Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [6]:
for x in ds:
    pass

In [7]:
x.shape

TensorShape([Dimension(160000), Dimension(1)])

In [8]:
def add_word(positives, negatives, negative_ratio):
    """
    Add words to background record
    :param words: word
    :return: new background with the word add randomly in it
    """

    def fn(background):

        if np.random.random() > negative_ratio:

            label, word_label_to_add = random.choice(list(enumerate(WORDS, 1)))
            word_to_add = random.choice(positives[word_label_to_add])

        else:
            word_to_add, label = random.choice(negatives), 0

        word_length = len(word_to_add)
        background_length = background.shape[0]

        start, end = get_random_time_segment(word_length, background_length)

        word_to_add = tf.convert_to_tensor(np.array(word_to_add), dtype=tf.float32)

        X = tf.concat([background[:start], word_to_add, background[end:]], axis=0)

        indices = list(enumerate([0] * start + [label] * (end - start) + [0] * (background_length - end)))

        Y = tf.SparseTensor(indices=indices, values=[1] * background_length,
                            dense_shape=[background_length, len(WORDS) + 1])

        return X, tf.sparse.to_dense(Y)

    return fn

In [9]:
ds = ds.map(add_word(positives, negatives, .3))

TypeError: __int__ returned non-int (type NoneType)